# Cleaning CSV

This is my attempt at cleaning up the [Goodreads dataset I found on Kaggle](https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks). My eventual goal is to tranform the dataset and upload it to a Neo4j database. But there is a lot of work to be done before that. Let's get started!

## Importing Libraries

the packages I'm including are fairly self explanatory, except for `FuzzyWuzzy` library, which i have not used before. `FuzzyWuzzy` is a Python library that provides tools for fuzzy string matching, allowing the user to compare strings that may have some differences or errors, such as typos or misspellings. It uses the Levenshtein Distance algorithm to calculate the difference between two strings, and returns a score between 0 and 100 that indicates how similar the strings are. I will be using this to clean up the `publisher` column.

In [ ]:
import pandas as pd
import re
import json
from fuzzywuzzy import fuzz, process

## Functions

These are the various functions I use throughout the notebook.

In [ ]:
def strip_spaces(x):
    """
    removes all leading, trailing, and extra spaces from a string
    """
    if isinstance(x, str):
        x = x.strip()
        x = re.sub('\s+', ' ', x)
        return x
    else:
        return x

In [ ]:
def validate_isbn10(isbn):
    """
    A function which determines if a given ISBN10 number is valid
    
    The rule for isbn's is that you multiply the first digit by 10, the second by 9, the third by 8th, ect.
    The result of this calculation should be a number divisible by 11.
    The final digit of an isbn may be an 'X'. This is to be interpreted as a 10 for purposes of calculation
    """
    result = 0
    
    # intially checks if given isbn is the right length and characters are valid
    if len(isbn) != 10:
        return False
    elif not isbn[0:8].isdigit():
        return False
    elif not isbn[-1].isdigit() or isbn[-1].lower() != 'x':
        return False
    
    # iterates through isbn
    for i in range(9):
        result += result + int(isbn[i]) * (10 - i)
    
    # Adding last character
    if isbn[9].lower() == 'x':
        result += 10
    else:
        result += int(isbn[9])
    
    # determine validity of isbn
    if result % 11 == 0:
        return True
    else:
        return False
    

In [ ]:
def validate_all_isbn(df):
    valid_count = 0
    invalid_count = 0

    for isbn in df['isbn_10']:
        if validate_isbn10(isbn):
            valid_count += 1
        else:
            invalid_count += 1

    print("Valid ISBN count:", valid_count)
    print("Invalid ISBN count:", invalid_count)

In [ ]:
def standardize_name(name, default_names, threshold=65):
    """
    compares a name to a list,
    if the name is similar enough it returns the best match,
    otherwise it keeps the name.
    """
    best_match, score = process.extractOne(name.lower(), default_names)
    if score >= threshold:
        return best_match
    else:
        return name

## Loading data

Intial attempt to load data fails.

In [ ]:
try:
    df = pd.read_csv('data/books_original_BROKEN.csv')
except Exception as e:
    print(e)

The issue in the authors column. Some of the authors names have split into the next cell to the right, and consequentially pushed everything else over as well. In order to fix this, I have to manually go into excel and fix line 3350. There are other rows with this issue, but for whatever reason the others don't prevent parsing of the dataframe. I create a copy of `books.csv` called `books_dirty.csv` to maintain the orginial dataset. The remaining work will be preformed on this copy. 

Additionally, when loading this dataframe I didn't change the encoding. So foreign characters were not appearing properly. Discovered proper encoding `utf-8-sig`

**Additionally ADDITIONALLY, I had been working for quite a while transforming the data before realizing that the isbn13 was not properly loading. The last six digits of each value was being replaced with zeros. I hunted around online and apparently I had to change it to *zipcodes* in excel for it to properly save the data. I don't know why this worked, and I'm not looking into it further.**

**ADDITIONALLY ADDITIONALLY ADDITIONALLY** further investigation of `isbn` shows that when I intially edited `books.csv`, by opening it in excel I altered a lot of the data. As a result of this, ISBN's that began with a `0` where truncated to exclude the leading zero. After opening the file in notepad and making the changes manually, many of these issues were fixed. So...I have now saved three seperate .csv's. `book_original_BROKEN`, `books_dirty.csv`, and `books_dirty_manual_fix.csv`.

### It turns out the real dirty data is the ones you make along the way.

In [ ]:
df = pd.read_csv('data/books_dirty.csv', dtype={'isbn13':str}, encoding='utf-8-sig') # Success!
df.head()

# Cleaning
## Columns

In [ ]:
df.info()

So there is this pesky extra column. Another issue created by things being pushed over is the datatype of columns that should be `int64` have been converted to string objects. I create a mask to see exactly which rows are creating the problem.

**ADDENDUM** after solving this problem manually in notepad, this code is no longer relevant. Keeping for posterity!

In [ ]:
try:
    mask = df['Unnamed: 12'].notnull()
    column_12_df = df[mask]
    print(column_12_df['Unnamed: 12'])
    print(df['average_rating'].unique())
except Exception as e:
    print(e)

Since some of the authors are now in the `average_rating` column, I find all the places where this a value that isn't an `int`. I then append these values to the `authors` column, shift everything over and then then delete the `Unnamed: 12` column.

**Again this is no longer an issue, but posterity!**

In [ ]:
# Create a boolean mask for non-numeric values in the 'average_rating' column
error_mask = pd.to_numeric(df['average_rating'], errors='coerce').isnull()

# Concatenate the 'authors' and 'average_rating' columns where there is an error
df.loc[error_mask, 'authors'] = df.loc[error_mask, 'authors'] + ' ' + df.loc[mask, 'average_rating']

# Shift all values to the right of the 'average_rating' column one cell to the left where there was an error
df.loc[error_mask, 'average_rating':'publisher'] = df.loc[error_mask, 'isbn':'Unnamed: 12'].values

#delete the Unnamed: 12 column
df = df.drop(columns=['Unnamed: 12'])

In [ ]:
print(df['average_rating'].unique())
df.info()

I now go through the columns and alter the datatypes. `average_rating` should be a float, `num_pages`, `ratings_count`, and `text_reviews_count` should all be integers, and the `publication_date` should be a date. I also clean up the titles of columns. `num_pages` obviously has some extra spaces, so I strip leading and trialing spaces. To be safe, I do this to all column titles.

In [ ]:
# cleaning column titles
df.columns = [strip_spaces(col) for col in df.columns]
df.info()
df.head()

In [ ]:
# transform column datatypes
df = df.astype({'average_rating': float, 'num_pages': int, 'ratings_count': int, 'text_reviews_count': int})
df.info()

no problems with the above transformations. Translating `publication_date` to datetime proves problematic.

In [ ]:
try:
    df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y')
except Exception as e:
    print(e)

Force converstion, transforming invalid dates to `NaT`. Find specific rows.

In [ ]:
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y', errors='coerce')
df[df['publication_date'].isna()]

Looking at the data in the excel spreadsheet, these two have publications on the 31st of months that don't have 31 days! Did a little research and found correct publication dates for both. double check they are correct datatype.

In [ ]:
df.loc[df['isbn'] == '553575104', 'publication_date'] = '10/31/2000'
df.loc[df['isbn'] == '2070323285', 'publication_date'] = '01/01/1975'
print(type(df.loc[8180, 'publication_date']))
print(type(df.loc[11098, 'publication_date']))

In [ ]:
df.info()

Now that all the existing columns are the correct datatype, I want to change the some of the titles. I also now strip all leading, trailing, and inbetween extra spaces in the dataset (before I forget)

In [ ]:
df = df.rename(columns={'bookID': 'book_id', 'isbn': 'isbn_10', 'isbn13': 'isbn_13'})
df = df.applymap(strip_spaces)
df.info()
df.head()

## ISBNs

Last semester when I was looking at the `isbn` and `isbn13` columns I felt they were largely redundant. I am now throughly convinced that they in fact AREN'T redudant. Which actually turns out to be larger issue. I start by checking the lengths of the values. `isbn` numbers should be 10 characters long, and `isbn13` should be 13 characters long.

In [ ]:
df['isbn_10'].str.len().value_counts()

In [ ]:
df['isbn_13'].str.len().value_counts()

Oh no! So there are obviously some issues, more so in the `isbn` column than the `isbn13` column. The first thing I'm going to check is how many of the ISBN's are valid. An ISBN10 is valid when the sum of multipling the first digit by 10, the second by 9, the third by 8th, ect. results in a number divisble by 11. Additionally, The final digit of an isbn may be an 'X'. This is to be interpreted as a 10 for purposes of calculation

In [ ]:
def validate_isbn10(isbn):
    """
    A function which determines if a given ISBN10 number is valid
    
    The rule for isbn's is that you multiply the first digit by 10, the second by 9, the third by 8th, ect.
    The result of this calculation should be a number divisible by 11.
    The final digit of an isbn may be an 'X'. This is to be interpreted as a 10 for purposes of calculation
    """
    result = 0
    
    # intially checks if given isbn is the right length and characters are valid
    if len(isbn) != 10:
        return False
    elif not isbn[0:8].isdigit():
        return False
    elif not isbn[-1].isdigit() and isbn[-1].lower() != 'x':
        return False
    
    # iterates through isbn
    for i in range(9):
        result += int(isbn[i]) * (10 - i)
    
    # Adding last character
    if isbn[9].lower() == 'x':
        result += 10
    else:
        result += int(isbn[9])
    
    # determine validity of isbn
    if result % 11 == 0:
        return True
    else:
        return False

In [ ]:
def validate_all_isbn(df):
    valid_count = 0
    invalid_count = 0

    for isbn in df['isbn_10']:
        if validate_isbn10(isbn):
            valid_count += 1
        else:
            invalid_count += 1

    print("Valid ISBN count:", valid_count)
    print("Invalid ISBN count:", invalid_count)

In [ ]:
validate_all_isbn(df)

So it looks like the majority of isbns that are the correct length are valid. let's take a look at those that are only a digit short. I have a hunch I want to check out. I'm thinking that it's possible that leading zeros were dropped from the orginal dataset during encoding.

In [ ]:
short_isbn = df[df['isbn_10'].str.len() == 9]['isbn_10']
df_short_isbn = short_isbn.to_frame()
df_short_isbn['isbn_10'] = '0' + df_short_isbn['isbn_10'].astype(str)

In [ ]:
and now check with the validity function

In [ ]:
validate_all_isbn(df_short_isbn)

Success!!! I now implement this change in the orginal dataframe. And check once more.

In [ ]:
df.loc[df['isbn_10'].str.len() == 9, 'isbn_10'] = '0' + df.loc[df['isbn_10'].str.len() == 9, 'isbn_10']

In [ ]:
validate_all_isbn(df)

### ADDENDUM:

so again, after the manual fix, this issue isn't a problem. When we load the manual fix file into the dataframe we get very different results.

In [ ]:
manual_fix_df = pd.read_csv('data/books_dirty_manual_fix.csv', dtype={'isbn13':str}, encoding='utf-8-sig')

manual_fix_df.columns = [strip_spaces(col) for col in manual_fix_df.columns]
manual_fix_df = manual_fix_df.astype({'average_rating': float, 'num_pages': int, 'ratings_count': int, 'text_reviews_count': int})
manual_fix_df = manual_fix_df.rename(columns={'bookID': 'book_id', 'isbn': 'isbn_10', 'isbn13': 'isbn_13'})
manual_fix_df = manual_fix_df.applymap(strip_spaces)

manual_fix_df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y', errors='coerce')
manual_fix_df.loc[df['isbn_10'] == '553575104', 'publication_date'] = '10/31/2000'
manual_fix_df.loc[df['isbn_10'] == '2070323285', 'publication_date'] = '01/01/1975'

manual_fix_df.info()

In [ ]:
manual_fix_df['isbn_10'].str.len().value_counts()

In [ ]:
manual_fix_df['isbn_13'].str.len().value_counts()

In [ ]:
validate_all_isbn(manual_fix_df)

I have learned a valuable lesson. That lesson being: **I both don't understand and deeply hate Excel.** With this lesson in mind, I will be going forward with the manual fix dataframe for the remainder of my editing.

In [ ]:
df = manual_fix_df

## Inserting audio column

It appears that there is no distinction made between text and audiobooks. This creates confusing data where "books" will single digit number of pages. I find distinct publishers and create two lists, one for regular books and one for audiobooks. Any publisher with "audio", "tape", "media", and "listen" in the name or that have 15 or less pages is added to the audio_publishers list.

In [ ]:
publishers = df['publisher'].unique()
audio_publishers = df[df['publisher'].str.contains('audio|tape|media|listen|caedmon', case=False) | (df['num_pages'] <= 15)]['publisher'].unique()
print_publishers = list(set(publishers) - set(audio_publishers))

print('list of print publishers:')
for pub in sorted(print_publishers):
    print('\t' + pub)

In [ ]:
print('list of audio publishers:')
for pub in sorted(audio_publishers):
    print('\t' + pub)

a lot of duplicates in both! This is something I'll need to fix, but for now I will use the lists to insert a new column `audio_book` which contains a boolean value denoting if said book is an audiofile, then my dataframe so that this column appears after the language_code.

In [ ]:
audio_mask = df["publisher"].isin(audio_publishers)
df['audio_book'] = audio_mask
column_order= ['book_id',
               'title',
               'authors',
               'isbn_10',
               'isbn_13',
               'language_code',
               'audio_book',
               'num_pages',
               'ratings_count',
               'average_rating',
               'text_reviews_count',
               'publication_date',
               'publisher']
df = df[column_order]
df.info()

## Publishers Cleanup

So there were a lot of duplicate values in the publisher column. Now admittedly, there are a lot of different imprints of publishers, but I think my eventual calculations in Neo4j will benefit from the simplification. I will be using the fuzzywuzzy package to help clean up some of the names.  and also `str.title()` method to normalize the names a little more.

In [ ]:
print(f'Number of Publishers: {len(df["publisher"].unique())}')

In [ ]:
def standardize_name(name, default_names, threshold=65):
    """compares a name to a list,
    if the name is similar enough it returns the best match,
    otherwise it keeps the name"""
    best_match, score = process.extractOne(name.lower(), default_names)
    if score >= threshold:
        return best_match
    else:
        return name

In [ ]:
df['publisher'] = df['publisher'].str.title()

publisher_counts = df['publisher'].value_counts().sort_values(ascending=False)

default_publishers = list(publisher_counts[publisher_counts >= 25].index)

df['publisher'] = df['publisher'].apply(standardize_name, args=(default_publishers,))

In [ ]:
print(f'Number of Publishers: {len(df["publisher"].unique())}')

In [ ]:
sorted(df['publisher'].unique())

So everything is looking good. We've chopped the list of publishers to a little over half of what it used to be. I'm a little tired of trying to automate everything (fuzzywuzzy was pain to figure out) so I decide to just roll up my sleeves and pare down the list more manually. I will mostly be using regex with a custom function to remove extra things I don't want, and the `df.loc` and `str.contains` methods to normalize larger publishers.

In [ ]:
def remove_regex_pattern(regex, column='publisher'):
    pattern = re.compile(regex)
    df[column] = df[column].str.replace(pattern, '')

In [ ]:
# Remove anything inside parenthesis
remove_regex_pattern(r'\s*\([^)]*\)')

# Remove Ltd
remove_regex_pattern(r'\bLtd\.?\b')

# Remove Inc and Inc.
remove_regex_pattern(r'\bInc\.?\b')

#Remove Publisher and Publishers
remove_regex_pattern(r'.Publisher(s)?\b')

# Remove wierd space plus period
remove_regex_pattern(r'\s\.')

# Remove 'Tb'
remove_regex_pattern(r'\sTb\.?')

# Remove Verlag
remove_regex_pattern(r'\sVerlag')

df['publisher'] = df['publisher'].str.replace(r'\s+And\s+', ' & ')
df['publisher'] = df['publisher'].str.replace(r'Pubns', '')

# Change Bbc to BBC because it was bothering me
df['publisher'] = df['publisher'].str.replace(r'Bbc', 'BBC')

# Melt Bloomsbury companies into single publisher
df.loc[df['publisher'].str.contains('Bloomsbury'), 'publisher'] = 'Bloomsbury Publishing'

# Same with Gallimard
df.loc[df['publisher'].str.contains('Gallimard'), 'publisher'] = 'Gallimard'

# Hachette
df.loc[df['publisher'].str.contains('Hachette'), 'publisher'] = 'Hachette'

# Harlequin
df.loc[df['publisher'].str.contains('Harlequin'), 'publisher'] = 'Harlequin'

# HarperCollins
df.loc[df['publisher'].str.contains('Harper'), 'publisher'] = 'HarperCollins'

# Headline
df.loc[df['publisher'].str.contains('Headline'), 'publisher'] = 'Headline'

# Kodansha
df.loc[df['publisher'].str.contains('Kodansha'), 'publisher'] = 'Kodansha'

# Macmillan
df.loc[df['publisher'].str.contains('Macmillan'), 'publisher'] = 'Macmillan'

# Macgraw-Hill
df.loc[df['publisher'].str.contains('Mcgraw-Hill'), 'publisher'] = 'Mcgraw-Hill'

# Penguin
df.loc[df['publisher'].str.contains('Penguin'), 'publisher'] = 'Penguin'

# Random House
df.loc[df['publisher'].str.contains('Random House'), 'publisher'] = 'Random House'

# Reclam
df.loc[df['publisher'].str.contains('Reclam'), 'publisher'] = 'Reclam'

# Silhouette
df.loc[df['publisher'].str.contains('Silhouette'), 'publisher'] = 'Silhouette'

# St. Martin's
df.loc[df['publisher'].str.contains('St. Martin'), 'publisher'] = "St. Martin's"

# Viking
df.loc[df['publisher'].str.contains('Viking'), 'publisher'] = 'Viking'

# Warner
df.loc[df['publisher'].str.contains('Warner'), 'publisher'] = 'Warner Books'

# Strip leading/trailing whitespaces I might have created
df['publisher'] = df['publisher'].str.strip()


In [ ]:
print(f'Number of Publishers: {len(df["publisher"].unique())}')

In [ ]:
sorted(df['publisher'].unique())

Finally, I go through the list and do the thing I was avoiding up till now, individually correcting/mapping publishers.

In [ ]:
publisher_mapping = {
    '18-Oct': 'Vintage',
    'Chatto Windus': 'Chatto & Windus',
    'Debols!Llo': 'Debolsillo',
    'Douglas Mcintyre': 'Douglas & Mcintyre',
    'Doubleday Canada': 'Doubleday Publishing',
    'Doubleday': 'Doubleday Publishing',
    'Dk': 'Dk Publishing',
    'Dk Children': 'Dk Publishing',
    'Emecé Editores': 'Emece Editores',
    'Faber & Faber Limited': 'Faber & Faber',
    'Faber Faber': 'Faber & Faber',
    'Fawcett Coventry': 'Fawcett',
    'Fawcett Crest': 'Fawcett',
    'Fasa Corp.': 'Fasa Corporation',
    'Five Star': 'Five Star Trade',
    'Fourth Estate Paperbacks': 'Fourth Estate',
    'Hill & Wang Publ.': 'Hill & Wang',
    'Hodder & Stoughton Educational Division': 'Hodder & Stoughton',
    'Inner Traditions International': 'Inner Traditions',
    'Insel Frankfurt': 'Insel',
    'John Wiley': 'John Wiley & Sons',
    'Limelight Editions': 'Limelight',
    'Liveright': 'Liveright Publishing Corp.',
    'Nal': 'New American Library',
    'Nal Jam': 'New American Library',
    'Nal Trade': 'New American Library',
    'National Geographic Society': 'National Geographic',
    'Plaza & Janes Editores Sa': 'Plaza & Janés',
    'Plaza & Janés Mexico': 'Plaza & Janés',
    'Plaza Y Janés': 'Plaza & Janés',
    'Prentice Hall Ptr': 'Prentice Hall',
    'Roc Hardcover': 'Roc',
    'Roc Trade': 'Roc',
    'Rowohlt Taschenbuch Gmbh': 'Rowohlt',
    'Schirmer Mosel': 'Schirmer/Mosel',
    'Simon Schuster': 'Simon & Schuster',
    'Thames Hudson': 'Thames & Hudson',
    'Ullstein Buchverlage Gmbh & Co. Kg / Ullstein Tas': 'Ullstein',
    'Virago Uk': 'Virago',
    'Vision Forum': 'Vision',
    'W. W. Norton Company': 'W. W. Norton & Company',
    'William Morrow Paperbacks': 'William Morrow',
    '小学館 [ShōGakukan]': '小学館'
}

df['publisher'] = df['publisher'].replace(publisher_mapping)

In [ ]:
len(df['publisher'].unique())

In [ ]:
df.reset_index(drop=True)
df['book_id'] = df.index + 1

df.head()

## Author and Publication Tables

Now that I've cleaned up the publishers, I'm ready to create seperate tables. I will be making both an `author` and a `publisher` table. The `author` table is necessary in order to properly extract names from the `authors` column of the our original dataset, without creating a lot of extra null values.

In [ ]:
# Split authors column and create a new dataframe
authors_df = df[['book_id', 'authors']].set_index('book_id').stack().str.split('/', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='author_name')

# Create author_id column
unique_authors = sorted(authors_df['author_name'].unique())
author_id = range(1, len(unique_authors) + 1)
authors_dict = {'author_name': unique_authors, 'author_id': author_id}
authors_id_df = pd.DataFrame(authors_dict)

authors_df = pd.merge(authors_df, authors_id_df, on='author_name')

# Reorder columns and rows
authors_df = authors_df[['author_id', 'author_name', 'book_id']]
authors_df = authors_df.sort_values(by='author_id')
authors_df = authors_df.reset_index(drop=True)
authors_df.head(30)

In [ ]:
Looks good! Now for the publishers. The `publication` table needs to be created in order for me to more easily implement my upload to neo4j. It will be pretty straight forward, with `publisher_id`, `publisher_name`, `publication_date`, and `book_id` foreign key. The only thing that really needs to be created is the `publisher_id`.

In [ ]:
# create publisher ids
unique_publishers = sorted(df['publisher'].unique())
publisher_id = range(1, len(unique_publishers) + 1)
publisher_dict = {'publisher_id': publisher_id, 'publisher': unique_publishers}
unique_publisher_df = pd.DataFrame(publisher_dict)

# create new table
publisher_df = df[['publisher', 'publication_date', 'book_id']]
publisher_df = pd.merge(publisher_df, unique_publisher_df, on='publisher')

# resturcture table
publisher_df = publisher_df[['publisher_id', 'publisher', 'publication_date', 'book_id']]
publisher_df = publisher_df.sort_values(by='publisher_id')
publisher_df = publisher_df.reset_index(drop=True)
publisher_df = publisher_df.rename(columns={'publisher': 'publisher_name'})
publisher_df.head(30)

## Wrapping up!

The final steps are removing redundant columns from my original dataframe and using `.to_csv` to export all my new tables.

In [ ]:
df = df.drop(['authors', 'publication_date', 'publisher'], axis=1)
df.head()

In [ ]:
df.to_csv('data/books_cleaned.csv', index=False)
authors_df.to_csv('data/authors.csv')
publisher_df.to_csv('data/publishers.csv')

# See neo4j_upload.ipynb for next steps!

In [128]:
df['publisher'] = df['publisher'].str.title()

publisher_counts = df['publisher'].value_counts().sort_values(ascending=False)

default_publishers = list(publisher_counts[publisher_counts >= 25].index)

df['publisher'] = df['publisher'].apply(standardize_name, args=(default_publishers,))

In [129]:
print(f'Number of Publishers: {len(df["publisher"].unique())}')

Number of Publishers: 872


In [130]:
sorted(df['publisher'].unique())

['10/18',
 '1St Book Library',
 'Aa World Services',
 'Abacus',
 'Abstract Studio',
 'Academy Chicago Publishers',
 'Ace',
 'Actes Sud',
 'Addison Wesley',
 'Addison-Wesley Professional',
 'Adv Manga',
 'Aegypan',
 'Aha! Process',
 'Ait Planet Lar',
 'Aladdin',
 'Albert Whitman Company',
 'Albin Michel',
 'Alcoholics Anonymous World Services Inc',
 'Alderac Entertainment Group',
 'Alderac Entertainment Group (Aeg)',
 'Alfabeta Bokförlag',
 'Alfaguara',
 'Alfaguara Infantil',
 'Alfred A. Knopf',
 'Alianza Editorial',
 'Allen & Ulwin',
 'Allen Lane',
 'Allworth',
 'Alpha',
 'Amadeus',
 'Amereon Limited',
 "America'S Test Kitchen",
 'American Diabetes Association',
 'Amistad',
 'Amr/Advanced Management Reports',
 'Anagrama',
 'Anchor',
 'Anchor Books',
 'Andrews And Mcmeel',
 'Apple',
 'Archaia',
 'Arden Shakespeare',
 'Aris & Phillips',
 'Aris And Phillips',
 'Arkana',
 'Arrow',
 'Arthur A. Levine',
 'Artisan',
 'Aspect',
 'Atheneum',
 'Audio Literature',
 'Audio Partners',
 'Audio Renai

So everything is looking good. We've chopped the list of publishers to a little over half of what it used to be. I'm a little tired of trying to automate everything (fuzzywuzzy was pain to figure out) so I decide to just roll up my sleeves and pare down the list more manually. I will mostly be using regex with a custom function to remove extra things I don't want, and the `df.loc` and `str.contains` methods to normalize larger publishers.

In [131]:
def remove_regex_pattern(regex, column='publisher'):
    pattern = re.compile(regex)
    df[column] = df[column].str.replace(pattern, '')

In [132]:
# Remove anything inside parenthesis
remove_regex_pattern(r'\s*\([^)]*\)')

# Remove Ltd
remove_regex_pattern(r'\bLtd\.?\b')

# Remove Inc and Inc.
remove_regex_pattern(r'\bInc\.?\b')

#Remove Publisher and Publishers
remove_regex_pattern(r'.Publisher(s)?\b')

# Remove wierd space plus period
remove_regex_pattern(r'\s\.')

# Remove 'Tb'
remove_regex_pattern(r'\sTb\.?')

# Remove Verlag
remove_regex_pattern(r'\sVerlag')

df['publisher'] = df['publisher'].str.replace(r'\s+And\s+', ' & ')
df['publisher'] = df['publisher'].str.replace(r'Pubns', '')

# Change Bbc to BBC because it was bothering me
df['publisher'] = df['publisher'].str.replace(r'Bbc', 'BBC')

# Melt Bloomsbury companies into single publisher
df.loc[df['publisher'].str.contains('Bloomsbury'), 'publisher'] = 'Bloomsbury Publishing'

# Same with Gallimard
df.loc[df['publisher'].str.contains('Gallimard'), 'publisher'] = 'Gallimard'

# Hachette
df.loc[df['publisher'].str.contains('Hachette'), 'publisher'] = 'Hachette'

# Harlequin
df.loc[df['publisher'].str.contains('Harlequin'), 'publisher'] = 'Harlequin'

# HarperCollins
df.loc[df['publisher'].str.contains('Harper'), 'publisher'] = 'HarperCollins'

# Headline
df.loc[df['publisher'].str.contains('Headline'), 'publisher'] = 'Headline'

# Kodansha
df.loc[df['publisher'].str.contains('Kodansha'), 'publisher'] = 'Kodansha'

# Macmillan
df.loc[df['publisher'].str.contains('Macmillan'), 'publisher'] = 'Macmillan'

# Macgraw-Hill
df.loc[df['publisher'].str.contains('Mcgraw-Hill'), 'publisher'] = 'Mcgraw-Hill'

# Penguin
df.loc[df['publisher'].str.contains('Penguin'), 'publisher'] = 'Penguin'

# Random House
df.loc[df['publisher'].str.contains('Random House'), 'publisher'] = 'Random House'

# Reclam
df.loc[df['publisher'].str.contains('Reclam'), 'publisher'] = 'Reclam'

# Silhouette
df.loc[df['publisher'].str.contains('Silhouette'), 'publisher'] = 'Silhouette'

# St. Martin's
df.loc[df['publisher'].str.contains('St. Martin'), 'publisher'] = "St. Martin's"

# Viking
df.loc[df['publisher'].str.contains('Viking'), 'publisher'] = 'Viking'

# Warner
df.loc[df['publisher'].str.contains('Warner'), 'publisher'] = 'Warner Books'

# Strip leading/trailing whitespaces I might have created
df['publisher'] = df['publisher'].str.strip()


C:\Users\mhous\AppData\Local\Temp\ipykernel_49724\3376952053.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  df['publisher'] = df['publisher'].str.replace(r'\s+And\s+', ' & ')


In [133]:
print(f'Number of Publishers: {len(df["publisher"].unique())}')

Number of Publishers: 771


In [134]:
sorted(df['publisher'].unique())

['10/18',
 '1St Book Library',
 'Aa World Services',
 'Abacus',
 'Abstract Studio',
 'Academy Chicago',
 'Ace',
 'Actes Sud',
 'Addison Wesley',
 'Addison-Wesley Professional',
 'Adv Manga',
 'Aegypan',
 'Aha! Process',
 'Ait Planet Lar',
 'Aladdin',
 'Albert Whitman Company',
 'Albin Michel',
 'Alcoholics Anonymous World Services',
 'Alderac Entertainment Group',
 'Alfabeta Bokförlag',
 'Alfaguara',
 'Alfaguara Infantil',
 'Alfred A. Knopf',
 'Alianza Editorial',
 'Allen & Ulwin',
 'Allen Lane',
 'Allworth',
 'Alpha',
 'Amadeus',
 'Amereon Limited',
 "America'S Test Kitchen",
 'American Diabetes Association',
 'Amistad',
 'Amr/Advanced Management Reports',
 'Anagrama',
 'Anchor',
 'Anchor Books',
 'Andrews & Mcmeel',
 'Apple',
 'Archaia',
 'Arden Shakespeare',
 'Aris & Phillips',
 'Arkana',
 'Arrow',
 'Arthur A. Levine',
 'Artisan',
 'Aspect',
 'Atheneum',
 'Audio Literature',
 'Audio Partners',
 'Audio Renaissance',
 'Audiogo',
 'Audiotext',
 'Augsburg Fortress Publishing',
 'Aurum',

Finally, I go through the list and do the thing I was avoiding up till now, individually correcting/mapping publishers.

In [135]:
publisher_mapping = {
    '18-Oct': 'Vintage',
    'Chatto Windus': 'Chatto & Windus',
    'Debols!Llo': 'Debolsillo',
    'Douglas Mcintyre': 'Douglas & Mcintyre',
    'Doubleday Canada': 'Doubleday Publishing',
    'Doubleday': 'Doubleday Publishing',
    'Dk': 'Dk Publishing',
    'Dk Children': 'Dk Publishing',
    'Emecé Editores': 'Emece Editores',
    'Faber & Faber Limited': 'Faber & Faber',
    'Faber Faber': 'Faber & Faber',
    'Fawcett Coventry': 'Fawcett',
    'Fawcett Crest': 'Fawcett',
    'Fasa Corp.': 'Fasa Corporation',
    'Five Star': 'Five Star Trade',
    'Fourth Estate Paperbacks': 'Fourth Estate',
    'Hill & Wang Publ.': 'Hill & Wang',
    'Hodder & Stoughton Educational Division': 'Hodder & Stoughton',
    'Inner Traditions International': 'Inner Traditions',
    'Insel Frankfurt': 'Insel',
    'John Wiley': 'John Wiley & Sons',
    'Limelight Editions': 'Limelight',
    'Liveright': 'Liveright Publishing Corp.',
    'Nal': 'New American Library',
    'Nal Jam': 'New American Library',
    'Nal Trade': 'New American Library',
    'National Geographic Society': 'National Geographic',
    'Plaza & Janes Editores Sa': 'Plaza & Janés',
    'Plaza & Janés Mexico': 'Plaza & Janés',
    'Plaza Y Janés': 'Plaza & Janés',
    'Prentice Hall Ptr': 'Prentice Hall',
    'Roc Hardcover': 'Roc',
    'Roc Trade': 'Roc',
    'Rowohlt Taschenbuch Gmbh': 'Rowohlt',
    'Schirmer Mosel': 'Schirmer/Mosel',
    'Simon Schuster': 'Simon & Schuster',
    'Thames Hudson': 'Thames & Hudson',
    'Ullstein Buchverlage Gmbh & Co. Kg / Ullstein Tas': 'Ullstein',
    'Virago Uk': 'Virago',
    'Vision Forum': 'Vision',
    'W. W. Norton Company': 'W. W. Norton & Company',
    'William Morrow Paperbacks': 'William Morrow',
    '小学館 [ShōGakukan]': '小学館'
}

df['publisher'] = df['publisher'].replace(publisher_mapping)

In [136]:
len(df['publisher'].unique())

731

In [137]:
df.reset_index(drop=True)
df['book_id'] = df.index + 1

df.head()

,book_id,title,authors,isbn_10,isbn_13,language_code,audio_book,num_pages,ratings_count,average_rating,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,0439785960,9780439785969,eng,False,652,2095690,4.57,27591,2006-09-16,Scholastic
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,0439358078,9780439358071,eng,False,870,2153167,4.49,29221,2004-09-01,Scholastic
2,3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,0439554896,9780439554893,eng,False,352,6333,4.42,244,2003-11-01,Scholastic
3,4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,043965548X,9780439655484,eng,False,435,2339585,4.56,36325,2004-05-01,Scholastic
4,5,Harry Potter Boxed Set Books 1-5 (Harry Potter...,J.K. Rowling/Mary GrandPré,0439682584,9780439682589,eng,False,2690,41428,4.78,164,2004-09-13,Scholastic


## Author and Publication Tables

Now that I've cleaned up the publishers, I'm ready to create seperate tables. I will be making both an `author` and a `publisher` table. The `author` table is necessary in order to properly extract names from the `authors` column of the our original dataset, without creating a lot of extra null values.

In [138]:
# Split authors column and create a new dataframe
authors_df = df[['book_id', 'authors']].set_index('book_id').stack().str.split('/', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='author_name')

# Create author_id column
unique_authors = sorted(authors_df['author_name'].unique())
author_id = range(1, len(unique_authors) + 1)
authors_dict = {'author_name': unique_authors, 'author_id': author_id}
authors_id_df = pd.DataFrame(authors_dict)

authors_df = pd.merge(authors_df, authors_id_df, on='author_name')

# Reorder columns and rows
authors_df = authors_df[['author_id', 'author_name', 'book_id']]
authors_df = authors_df.sort_values(by='author_id')
authors_df = authors_df.reset_index(drop=True)
authors_df.head(30)

,author_id,author_name,book_id
0,1,A. Bartlett Giamatti,8140
1,2,A. Elizabeth Delany,4761
2,3,A. Merritt,6263
3,4,A. Roger Merrill,9220
4,5,A. Walton Litz,6190
5,6,A.B. Yehoshua,7173
6,6,A.B. Yehoshua,8041
7,7,A.D.P. Briggs,4955
8,8,A.E. Cunningham,10115
9,9,A.E. van Vogt,10117


In [ ]:
Looks good! Now for the publishers. The `publication` table needs to be created in order for me to more easily implement my upload to neo4j. It will be pretty straight forward, with `publisher_id`, `publisher_name`, `publication_date`, and `book_id` foreign key. The only thing that really needs to be created is the `publisher_id`.

In [139]:
# create publisher ids
unique_publishers = sorted(df['publisher'].unique())
publisher_id = range(1, len(unique_publishers) + 1)
publisher_dict = {'publisher_id': publisher_id, 'publisher': unique_publishers}
unique_publisher_df = pd.DataFrame(publisher_dict)

# create new table
publisher_df = df[['publisher', 'publication_date', 'book_id']]
publisher_df = pd.merge(publisher_df, unique_publisher_df, on='publisher')

# resturcture table
publisher_df = publisher_df[['publisher_id', 'publisher', 'publication_date', 'book_id']]
publisher_df = publisher_df.sort_values(by='publisher_id')
publisher_df = publisher_df.reset_index(drop=True)
publisher_df = publisher_df.rename(columns={'publisher': 'publisher_name'})
publisher_df.head(30)

,publisher_id,publisher_name,publication_date,book_id
0,1,10/18,2001-02-15,6904
1,1,10/18,1998-11-18,7329
2,2,1St Book Library,2003-01-14,8093
3,3,Aa World Services,2002-02-10,926
4,4,Abacus,1997-03-03,2069
5,4,Abacus,2005-01-20,8768
6,4,Abacus,2004-02-05,8656
7,4,Abacus,1989-02-01,1756
8,4,Abacus,1995-11-12,10583
9,4,Abacus,2000-04-01,1897


## Wrapping up!

The final steps are removing redundant columns from my original dataframe and using `.to_csv` to export all my new tables.

In [140]:
df = df.drop(['authors', 'publication_date', 'publisher'], axis=1)
df.head()

,book_id,title,isbn_10,isbn_13,language_code,audio_book,num_pages,ratings_count,average_rating,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,0439785960,9780439785969,eng,False,652,2095690,4.57,27591
1,2,Harry Potter and the Order of the Phoenix (Har...,0439358078,9780439358071,eng,False,870,2153167,4.49,29221
2,3,Harry Potter and the Chamber of Secrets (Harry...,0439554896,9780439554893,eng,False,352,6333,4.42,244
3,4,Harry Potter and the Prisoner of Azkaban (Harr...,043965548X,9780439655484,eng,False,435,2339585,4.56,36325
4,5,Harry Potter Boxed Set Books 1-5 (Harry Potter...,0439682584,9780439682589,eng,False,2690,41428,4.78,164


In [141]:
df.to_csv('data/books_cleaned.csv', index=False)
authors_df.to_csv('data/authors.csv')
publisher_df.to_csv('data/publishers.csv')

# See neo4j_upload.ipynb for next steps!